# 0.Overview
**Edit ver 2**:

For a simple tutorial, this notebook use test data with `full_sort_topk` function. Due to its limit, we are missing last item. So i create new notebooks with custom code, in order to use all items for submitted recommendation:
* Detail about `full_sort_topk` and custom function: https://www.kaggle.com/astrung/recbole-using-all-items-for-prediction
* New custom function with interaction data: https://www.kaggle.com/astrung/sequential-model-fixed-missing-last-item
* New custom function with interaction data + item features: https://www.kaggle.com/code/astrung/lstm-model-with-item-infor-fix-missing-last-item/notebook

Please check new notebook for better score and upvote if it help

**Edit ver 1**:

I have created a new version for adding user features in sequential model. It looks like by using item features, sequential get more accurate( i got higher score). If you want to improve your sequential model, please check my new notebook: https://www.kaggle.com/astrung/recbole-lstm-sequential-with-item-features


This notebook demonstrate how to use LSTM for recomendation system.
I am using Recbole as an open source, as it has so many built-in models for recommendation(CNN, GRU-LSTM, Context-aware, Graph). In this notebook, we tried to use GRU/LSTM model for testing effect of sequential model for recommendation.

Due to memory limit and faster testing purpose, we will just use data in 2020.

If you want to use with all of interactions in all time, i have created a new atomic dataset here for you: https://www.kaggle.com/astrung/hm-atomic-interation

We also have other limit: we only train model and predict with users who buy more than 40 items and items which is bought by more than 40 people.

We will follow below steps for creating model:

1. In order to use Recbole, we create atomic file from interaction data
2. Because we only use Recbole model for predicting with users who buy more than 40 items, other users will need to fill by default recomendation items. We create most viewed items in last month as defautl recomendation
3. We create dataset and train model in recbole.
4. We create prediction result by trained model
5. We combine recomendation result from most viewed items in last month and Recbole predicted model.

I will explain more detail in following cells.



In [ ]:
!pip install recbole

# 1. Create atomic file

In [ ]:
import pandas as pd
df = pd.read_csv(r"/kaggle/input/h-and-m-personalized-fashion-recommendations/transactions_train.csv", 
                 dtype={'article_id': 'str'})
df.head()

In [ ]:
df['t_dat'] = pd.to_datetime(df['t_dat'], format="%Y-%m-%d")
df

In [ ]:
import numpy as np
df['timestamp'] = df.t_dat.values.astype(np.int64) // 10 ** 9
df.head()

**We fill with data in only 2020(timestapm > > 1585620000) and create inter file**
For anyone need instruction about inter file, please check below links:
* https://recbole.io/docs/user_guide/data_intro.html
* https://recbole.io/docs/user_guide/data/atomic_files.html

In [ ]:
temp = df[df['timestamp'] > 1585620000][['customer_id', 'article_id', 'timestamp']].rename(
    columns={'customer_id': 'user_id:token', 'article_id': 'item_id:token', 'timestamp': 'timestamp:float'})
temp

We save atomic file in dataset format for using with recbole

In [ ]:
!mkdir /kaggle/working/recbox_data
temp.to_csv('/kaggle/working/recbox_data/recbox_data.inter', index=False, sep='\t')

# 2. We create defautl recomendation for user who can not be predicted by sequential model.
I use this approach in notebook: https://www.kaggle.com/hervind/h-m-faster-trending-products-weekly You can check it for more detail information. I will juse copy only code here

In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
sub0 = pd.read_csv('../input/hm-pre-recommendation/submissio_byfone_chris.csv').sort_values('customer_id').reset_index(drop=True)
sub1 = pd.read_csv('../input/hm-pre-recommendation/submission_trending.csv').sort_values('customer_id').reset_index(drop=True)
sub2 = pd.read_csv('../input/hm-pre-recommendation/submission_exponential_decay.csv').sort_values('customer_id').reset_index(drop=True)

sub0.shape, sub1.shape, sub2.shape

In [ ]:
sub0.columns = ['customer_id', 'prediction0']
sub0['prediction1'] = sub1['prediction']
sub0['prediction2'] = sub2['prediction']
del sub1, sub2
sub0.head()

In [ ]:
def cust_blend(dt, W = [1,1,1]):
    #Global ensemble weights
    #W = [1.15,0.95,0.85]
    
    #Create a list of all model predictions
    REC = []
    REC.append(dt['prediction0'].split())
    REC.append(dt['prediction1'].split())
    REC.append(dt['prediction2'].split())
    
    #Create a dictionary of items recommended. 
    #Assign a weight according the order of appearance and multiply by global weights
    res = {}
    for M in range(len(REC)):
        for n, v in enumerate(REC[M]):
            if v in res:
                res[v] += (W[M]/(n+1))
            else:
                res[v] = (W[M]/(n+1))
    
    # Sort dictionary by item weights
    res = list(dict(sorted(res.items(), key=lambda item: -item[1])).keys())
    
    # Return the top 12 itens only
    return ' '.join(res[:12])

sub0['prediction'] = sub0.apply(cust_blend, W = [1.05,1.00,0.95], axis=1)
sub0.head()

In [ ]:
del sub0['prediction0']
del sub0['prediction1']
del sub0['prediction2']
sub0.to_csv(f'submission.csv', index=False)

In [ ]:
del sub0
del df

# 3. Create dataset and train model with Recbole

For anyone need instruction document, please check this link: https://recbole.io/docs/user_guide/usage/use_modules.html

In [ ]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger

In [ ]:
parameter_dict = {
    'data_path': '/kaggle/working',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'user_inter_num_interval': "[30,inf)",
    'item_inter_num_interval': "[40,inf)",
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp']},
    'neg_sampling': None,
    'epochs': 50,
    'eval_args': {
        'split': {'RS': [9, 0, 1]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'}
}

config = Config(model='GRU4Rec', dataset='recbox_data', config_dict=parameter_dict)

# init random seed
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()
# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

# write config info into log
logger.info(config)

In [ ]:
dataset = create_dataset(config)
logger.info(dataset)

In [ ]:
# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

In [ ]:
# model loading and initialization
model = GRU4Rec(config, train_data.dataset).to(config['device'])
logger.info(model)

# trainer loading and initialization
trainer = Trainer(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(train_data)

# 4. Create recommendation result from trained model

I note document here for any one want to customize it: https://recbole.io/docs/user_guide/usage/case_study.html

In [ ]:
from recbole.utils.case_study import full_sort_topk
external_user_ids = dataset.id2token(
    dataset.uid_field, list(range(dataset.user_num)))[1:]#fist element in array is 'PAD'(default of Recbole) ->remove it 

In [ ]:
topk_items = []
for internal_user_id in list(range(dataset.user_num))[1:]:
    _, topk_iid_list = full_sort_topk([internal_user_id], model, test_data, k=12, device=config['device'])
    last_topk_iid_list = topk_iid_list[-1]
    external_item_list = dataset.id2token(dataset.iid_field, last_topk_iid_list.cpu()).tolist()
    topk_items.append(external_item_list)
print(len(topk_items))

In [ ]:
external_item_str = [' '.join(x) for x in topk_items]
result = pd.DataFrame(external_user_ids, columns=['customer_id'])
result['prediction'] = external_item_str
result.head()

# 5. Combine result from most bought items and GRU model

In [ ]:
submit_df = pd.read_csv('submission.csv')
submit_df.shape

In [ ]:
submit_df.head()

In [ ]:
submit_df = pd.merge(submit_df, result, on='customer_id', how='outer')
submit_df.head()

In [ ]:
submit_df = submit_df.fillna(-1)
submit_df['prediction'] = submit_df.apply(
    lambda x: x['prediction_y'] if x['prediction_y'] != -1 else x['prediction_x'], axis=1)
submit_df.head()

In [ ]:
submit_df[submit_df['prediction_y'] != -1]

In [ ]:
submit_df = submit_df.drop(columns=['prediction_y', 'prediction_x'])
submit_df.head()

In [ ]:
submit_df.to_csv('submission.csv', index=False)